In [1]:
!nvidia-smi

Wed May 11 14:07:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
import sys
drive.mount('/content/drive')
#设置路径
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

Mounted at /content/drive


In [3]:
! pip install -U synonyms

     |████████████████████████████████| 10.7 MB 2.9 MB/s 
  Created wheel for synonyms: filename=synonyms-3.16.0-py3-none-any.whl size=10832785 sha256=1d24686a500e5dc7cebabb23198a2b570cb81dd52c670bceec68e4c4cf512896
  Stored in directory: /root/.cache/pip/wheels/e4/cd/43/b4548753509a94471fc946967a07116252d49aeeb689db8f7c
Successfully built synonyms


In [4]:
import synonyms

[jieba] default dict file path ../data/vocab.txt
[jieba] default dict file path ../data/vocab.txt
[jieba] load default dict ../data/vocab.txt ...
[jieba] load default dict ../data/vocab.txt ...
>> Synonyms load wordseg dict [/usr/local/lib/python3.7/dist-packages/synonyms/data/vocab.txt] ... 
>> Synonyms on loading stopwords [/usr/local/lib/python3.7/dist-packages/synonyms/data/stopwords.txt] ...
[Synonyms] on loading vectors [/usr/local/lib/python3.7/dist-packages/synonyms/data/words.vector.gz] ...

[Synonyms] downloading data from https://github.com/chatopera/Synonyms/releases/download/3.15.0/words.vector.gz to /usr/local/lib/python3.7/dist-packages/synonyms/data/words.vector.gz ... 
 this only happens if SYNONYMS_WORD2VEC_BIN_URL_ZH_CN is not present and Synonyms initialization for the first time. 
 It would take minutes that depends on network.

[Synonyms] downloaded.



In [5]:
synonyms.nearby('篮球')

(['篮球', '橄榄球', '排球', '棒球', '美式足球', '冰球', '拳击', '网球', '高尔夫球', '高球'],
 [1.0,
  0.81482756,
  0.78554475,
  0.7845952,
  0.7815255,
  0.7550466,
  0.7411452,
  0.7350726,
  0.72586256,
  0.7199612])

# EDA(Easy Data Augmentation)
![UDA1](https://img-blog.csdnimg.cn/c5ffcca4482c4c42beb6f1215e37657c.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)
### EDA：用于提高文本分类任务性能的简单数据增强技术。 EDA 由四个简单但功能强大的操作组成：同义词替换、随机插入、随机交换和随机删除。
### 之前的工作已经提出了一些 NLP 中数据增强的技术，回译通过将句子翻译成法语然后再翻译成英语来生成新数据。

### 对于训练集中的给定句子，随机选择并执行以下操作之一：
* 同义词替换（SR）：从句子中随机选择 n 个不是停用词的词。 用随机选择的同义词之一替换这些单词中的每一个。
* 随机插入 (RI)：在句子中随机找到一个词，并找出其同义词，且该同义词不是停用词。 将该同义词插入句子中的随机位置。 这样做n次。
* 随机交换（RS）：随机选择句子中的两个单词并交换它们的位置。 这样做n次。
* 随机删除（RD）：以概率 p 随机删除句子中的每个单词。

### 停用词 stop word

停用词是指在信息检索中，为节省存储空间和提高搜索效率，在处理自然语言数据（或文本）之前或之后会自动过滤掉某些字或词，这些字或词即被称为Stop Words（停用词）
```
str = "00000003210Runoob01230000000"; 
print str.strip( '0' );  # 去除首尾字符 0

str2 = "   Runoob      ";   # 去除首尾空格
print str2.strip();

3210Runoob0123
Runoob
```


In [7]:
# strip() 方法用于移除字符串头尾指定的字符（默认为空格或换行符）或字符序列。
stop_words = {word.strip() for word in open('/content/drive/MyDrive/Colab Notebooks/baidu_stopwords.txt', 'r', encoding='utf8').readlines()}

In [8]:
# 同义词替换（SR）：从句子中随机选择 n 个不是停用词的词。 用随机选择的同义词之一替换这些单词中的每一个。
def get_synonyms(word):
  # (['mama'],['0.9'])
  #取出元组第0个元素（['mama']），并去重
  sys = set(synonyms.nearby(word)[0])
  #将word从同义词列表中去除
  if word in sys:
    sys.remove(word)
  return list(sys)
  # 如果输入"给力" 可能没有同义词（同义词只有他自己） 则返回  ([],[])


def synonym_replacement(words, n):
  new_words = words.copy()
  # 遍历句子中的每个词，并且这个词不是停用词
  # set()去重， 以列表形式返回
  random_word_list = list(set([word for word in words if word not in stop_words]))
  # 打乱
  random.shuffle(random_word_list)
  num_replaced = 0
  for random_word in random_word_list:
    # 得到近义词列表
    synonyms = get_synonyms(random_word)
    if len(synonyms) >= 1:
      # 随机取出一个同义词
      synonym = random.choice(list(synonyms))
      # 用synonym替换原词
      new_words = [synonym if word == random_word else word for word in new_words]
      num_replaced += 1
    if num_replaced >= n:
      break
  # new_words 已经是个列表了 下一块代码举例
  sentence = ' '.join(new_words)
  new_words = sentence.split(' ')
  return new_words

### 存在一种情况  
*有近义词：**actor** -> **film star** 一个单词的近义词是两个单词*

sentence = ['in', 'actor']

*默认new_words = ['in', 'film star']*

*希望有如下表示：*
new_words = ['in', 'film', 'star']

In [9]:
# 随机删除（RD）：以概率 p 随机删除句子中的每个单词。
def random_deletion(words, p):
  if len(words) == 1:
    return words
  new_words = []
  for word in words:
    # 概率
    r = random.uniform(0, 1)
    # r>p 保留
    if r > p:
      new_words.append(word)
  # 如果都删没了 随机返回一个词
  if len(new_words) == 0:
    random_int = random.randint(0, len(words) - 1)
    return [words[random_int]]
  return new words

SyntaxError: ignored

In [10]:
# 随机交换（RS）：随机选择句子中的两个单词并交换它们的位置。 这样做n次。
def swap_word(new_words):
  random_idx_1 = random.randint(0, len(new_words) - 1)
  random_idx_2 = random_idx_1
  
  count = 0
  # 两者相等重新取idx_2
  while random_idx_2 == random_idx_1:
    random_idx_2 - random.randint(0, len(new_words) - 1)
    count += 1
    # 取了三次还是相等 立即推
    if count > 3:
      return new_words

  new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1] 
  return new_words

def random_swap(words, n):
  new_words = words.copy()
  for _ in range(n):
    new_words = swap_word(new_words)
  return new_words